In [1]:
#loading the data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
pune_tmax_train = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_train_clean.csv')
Nex_train = pd.read_csv(r'D:\IITM\prepared data\Nex_train_ready.csv')

pune_tmax_test = pd.read_csv(r'D:\IITM\converted_to_csv\imd5p5_tmax_pune_test.csv')
Nex_test = pd.read_csv(r'D:\IITM\prepared data\Nex_test_ready.csv')

#using pune tmax values as labels 
tmax_labels = pd.DataFrame(pune_tmax_train,columns=["tmax"])
tmax_labels.head() # .head() function gives out first 5 rows of the datset 
nex_pr_train = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_pr_train.csv')
nex_pr_test = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_pr_test.csv')
Nex_train_final = pd.concat([Nex_train.reset_index(drop=True), nex_pr_train["precipitation"].reset_index(drop=True)],axis=1) 
Nex_test_final = pd.concat([Nex_test.reset_index(drop=True), nex_pr_test["precipitation"].reset_index(drop=True)],axis=1)
test_labels = pd.DataFrame(pune_tmax_test,columns=["tmax"])
test_labels.head()
nex_test_raw = pd.read_csv(r'D:\IITM\converted_to_csv\nex25_tmax_pune_test.csv')
nex_tmax = pd.DataFrame(nex_test_raw["nex_tmax"], columns=['nex_tmax'])

In [2]:
#RNN with min max scaling
from sklearn.preprocessing import MinMaxScaler
mm_X = MinMaxScaler()
mm_y = MinMaxScaler()
X_train = mm_X.fit_transform(Nex_train_final)
Y_train = mm_y.fit_transform(tmax_labels)
X_test = mm_X.fit_transform(Nex_test_final)
Y_test = mm_y.fit_transform(test_labels)

In [3]:
x_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
print(x_train.shape)
x_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
print(x_test.shape)

(10950, 4, 1)
(3650, 4, 1)


In [4]:
from keras.layers import Flatten
regressor = Sequential()
regressor.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(Dropout(0.2))
regressor.add(Flatten())
regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.add(Flatten())
regressor.add(Dense(units=1))
regressor.fit(x_train, Y_train, epochs=100, batch_size=32)

Epoch 1/100
343/343 [==============================] - 22s 3ms/step - loss: 0.0546
Epoch 2/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0188
Epoch 3/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0145
Epoch 4/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0139
Epoch 5/100
343/343 [==============================] - 1s 4ms/step - loss: 0.0133
Epoch 6/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0135A: 1
Epoch 7/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0135
Epoch 8/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0130
Epoch 9/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0134
Epoch 10/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0128
Epoch 11/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0129
Epoch 12/100
343/343 [==============================] - 1s 3ms/step - loss: 0.0125
Epoch 13

343/343 [==============================] - 3s 10ms/step - loss: 0.0108
Epoch 100/100
343/343 [==============================] - 3s 9ms/step - loss: 0.0110


In [5]:
y_test_pred_0 = regressor.predict(x_test)
rnn_test_pred_0 = mm_y.inverse_transform(y_test_pred_0)

In [6]:
#calculating RMSE over train set for SVR
from sklearn.metrics import mean_squared_error
rnn_mse_test = mean_squared_error(test_labels, rnn_test_pred_0)
rnn_rmse_test_0 = np.sqrt(rnn_mse_test)
print("RMSE for Support Vector Regression over test set = ")
rnn_rmse_test_0

RMSE for Support Vector Regression over test set = 


1.9604228761659506